## Gymnasium

In [2]:
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
from torch.distributions.normal import Normal
import gymnasium as gym
from gymnasium.wrappers import FlattenObservation
from skrl.models.torch.deterministic import DeterministicMixin
from skrl.models.torch import Model
from skrl.memories.torch import RandomMemory
from skrl.agents.torch.ddpg import DDPG, DDPG_DEFAULT_CONFIG


### Blackjack

In [3]:
env = gym.make('Blackjack-v1', natural=False, sab=False)

In [77]:
class DeterministicActor(DeterministicMixin, Model):
    def __init__(self, num_of_states, observation_space, action_space, device="cpu", clip_acitons=False):
        Model.__init__(self, observation_space, action_space, device)
        DeterministicMixin.__init__(self, clip_acitons)
        
        # if isinstance(observation_space, gym.spaces.Tuple):
        #     self.observation_spaces = observation_space.spaces
        #     self.num_observations = sum([space.shape[0] for space in self.observation_spaces])
        # else:
        #     raise ValueError("Observation space must be of type gym.spaces.Tuple")
        self.num_of_states = num_of_states
        self.linear_layer_1 = nn.Linear(self.num_observations, 32)
        self.linear_layer_2 = nn.Linear(32, 16)
        self.action_layer = nn.Linear(16, self.num_actions)
    
    def compute(self, inputs, role):
        print(inputs)
        initial_tensor = [0] * self.num_observations
        initial_tensor[inputs["states"][0][0]] = 1
        initial_tensor[inputs["states"][0][1] + self.num_of_states[0]] = 1
        initial_tensor[inputs["states"][0][2] + self.num_of_states[1]] = 1
        tensor = torch.tensor(initial_tensor)
        print(tensor)
        x = nn.functional.relu(self.linear_layer_1(tensor))
        x = nn.functional.relu(self.linear_layer_2(x))
        return torch.sigmoid(self.action_layer(x)), {}

In [78]:
class DeterministicCritic(DeterministicMixin, Model):
    def __init__(self, observation_space, action_space, device="cpu", clip_acitons=False):
        Model.__init__(self, observation_space, action_space, device)
        DeterministicMixin.__init__(self, clip_acitons)
        
        self.linear_layer_1 = nn.Linear(self.num_observations + self.num_actions, 32)
        self.linear_layer_2 = nn.Linear(32, 16)
        self.linear_layer_3 = nn.Linear(16, 1)
    
    def compute(self, inputs, role):
        print(inputs)
        x = nn.functional.relu(self.linear_layer_1(torch.cat([inputs["states"][0], inputs["taken_action"]], dim=1)))
        x = nn.functional.relu(self.linear_layer_2(x))
        return self.linear_layer_3(x), {}

In [79]:
memory = RandomMemory(memory_size=1000)

In [80]:
models = {}
models["policy"] = DeterministicActor((32, 11, 2), gym.spaces.flatten_space(env.observation_space), env.action_space)
models["target_policy"] = DeterministicActor((32, 11, 2), gym.spaces.flatten_space(env.observation_space), env.action_space)
models["critic"] = DeterministicCritic(gym.spaces.flatten_space(env.observation_space), env.action_space)
models["target_critic"] = DeterministicCritic(gym.spaces.flatten_space(env.observation_space), env.action_space)

In [81]:
for model in models.values():
    model.init_parameters(method_name="normal_", mean=0.0, std=0.1)

In [82]:
cfg_agent = DDPG_DEFAULT_CONFIG.copy()
cfg_agent["discount_factor"] = 0.9
agent = DDPG(models=models,
             memory=memory,  # only required during training
             cfg=cfg_agent,
             observation_space=env.observation_space,
             action_space=env.action_space,
             device="cpu")

In [83]:
num_episodes = 1000  # Set the number of episodes for training
for episode in range(num_episodes):
    obs = env.reset()
    done = False
    episode_reward = 0
    timestep = 0 
    while not done:
        action = agent.act(obs, timestep, num_episodes)
        next_obs, reward, done, _ = env.step(action)
        episode_reward += reward
        agent.replay_buffer.push(obs, action, reward, next_obs, done)
        obs = next_obs
        agent.update()
        timestep += timestep
    print(f"Episode {episode + 1}: Total Reward: {episode_reward}")

{'states': ((7, 9, 0), {})}
tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])


RuntimeError: expected scalar type Long but found Float

In [29]:
env.reset()

((12, 1, 0), {})

In [65]:
env.observation_space

Tuple(Discrete(32), Discrete(11), Discrete(2))

In [68]:
x = gym.spaces.flatten_space(env.observation_space)

AttributeError: 'Box' object has no attribute 'count'